In [2]:
import numpy as np
# pip install mne
import numpy as np
import mne
import pandas as pd
import scipy
from matplotlib import pyplot as plt 
import xml.etree.cElementTree as et
import tensorflow as tf
import librosa

# Define example data
window_size = 100
num_samples = 5
channels_A = 1
channels_B = 1
channels_C = 2

# Generate random data for each modality
data_A = np.random.randn(num_samples, window_size, channels_A)
data_B = np.random.randn(num_samples, window_size, channels_B)
data_C = np.random.randn(num_samples, window_size, channels_C)
data=[data_A,data_B,data_C]

# Printing out the shape of each modality's data
print(f"Modality A Data Shape: {data_A.shape}")
print(f"Modality B Data Shape: {data_B.shape}")
print(f"Modality C Data Shape: {data_C.shape}")


C:\Users\rafae\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Modality A Data Shape: (5, 100, 1)
Modality B Data Shape: (5, 100, 1)
Modality C Data Shape: (5, 100, 2)


In [3]:
df = pd.read_csv('00000999-100507_Flow Patient-0.csv')
df3 = pd.read_csv('00000999-100507_SpO2.csv')
df2 = pd.DataFrame(np.random.randn(20000, 2))

In [4]:
def create_dataset(data, window_size,samp_rate):
    windows = []
    window_size=window_size*samp_rate
    for i in range(0, data.shape[0], window_size):
        window = data[i:i+window_size]
        if len(window) == window_size:  # Discard incomplete windows (if any)
            windows.append(window)
    return np.array(windows)


In [5]:
ch1 = create_dataset(df,30,100)
ch2 = create_dataset(df3,30,1)

In [6]:
ch1.shape

(496, 3000, 1)

In [7]:
ch2.shape

(496, 30, 1)

In [8]:
def get_ts_modality_encoder(input_shape, modality_name,filters,code_size,l2_rate):
    initializer = tf.keras.initializers.RandomNormal(mean=0., stddev=1.)
    input = tf.keras.layers.Input(input_shape)
    x = tf.keras.layers.Conv1D(filters=2 * filters,
                               kernel_size=10,
                               activation="linear",
                               padding="same",
                               strides=1,
                               kernel_regularizer=tf.keras.regularizers.l2(l2_rate),
                               kernel_initializer=initializer)(input)

    x = tf.keras.layers.LayerNormalization()(x)
    x = tf.keras.layers.PReLU(shared_axes=[1])(x)

    x = tf.keras.layers.Conv1D(filters=filters,
                               kernel_size=8,
                               activation="linear",
                               padding="same",
                               strides=1,
                               kernel_regularizer=tf.keras.regularizers.l2(l2_rate),
                               kernel_initializer=initializer)(x)
    x = tf.keras.layers.LayerNormalization()(x)
    x = tf.keras.layers.PReLU(shared_axes=[1])(x)

    x = tf.keras.layers.Conv1D(filters=code_size,
                               kernel_size=4,
                               activation="linear",
                               padding="same",
                               strides=1,
                               kernel_regularizer=tf.keras.regularizers.l2(l2_rate),
                               kernel_initializer=initializer)(x)
    # output = tf.keras.layers.LayerNormalization()(x)
    x = tf.keras.layers.PReLU(shared_axes=[1])(x)
    output = tf.keras.layers.BatchNormalization(epsilon=1e-6)(x)

    return tf.keras.models.Model(input, output, name=modality_name)

In [9]:
# modality specific encoders
mod_encoder = []
mod_input = []

input_shape = (ch1.shape[1], ch1.shape[2])
print(input_shape)
encoder = get_ts_modality_encoder(input_shape,
                                  modality_name='flow',
                                  filters=24,
                                  code_size=256,
                                  l2_rate=1e-4)

mod_input.append(tf.keras.layers.Input(shape=input_shape))

x_a = encoder(mod_input[-1])
x_a = tf.keras.layers.GlobalMaxPooling1D()(x_a)
x_a = tf.keras.layers.Dense(256, activation="linear")(x_a)
mod_encoder.append(x_a)

input_shape = (ch2.shape[1], ch2.shape[2])
encoder = get_ts_modality_encoder(input_shape,
                                  modality_name='spo2',
                                  filters=24,
                                  code_size=256,
                                  l2_rate=1e-4)

mod_input.append(tf.keras.layers.Input(shape=input_shape))

x_a = encoder(mod_input[-1])
x_a = tf.keras.layers.GlobalMaxPooling1D()(x_a)
x_a = tf.keras.layers.Dense(256, activation="linear")(x_a)
mod_encoder.append(x_a)
# print(mod_input)
embedding_model = tf.keras.Model(mod_input, mod_encoder)
# input_x=embedding_model.input
# print(input_x)
# input_x = ([(ch1.shape[1], ch1.shape[2]),(ch2.shape[1], ch2.shape[2])])
# xi=embedding_model(input_x)
# x = tf.keras.layers.Concatenate()(embedding_model.output)
# x = tf.keras.layers.Flatten()(x)

# x = tf.keras.layers.Dense(128, activation="relu")(x)

# # Final classification layer
# output = tf.keras.layers.Dense(1, activation="sigmoid")(x)

# # Define the new model
# classifier_model = tf.keras.models.Model(inputs=embedding_model.inputs, outputs=output)
# classifier_model.summary()
embedding_model.summary()

(3000, 1)


C:\Users\rafae\anaconda3\lib\site-packages\keras\src\initializers\initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 3000, 1)]            0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 30, 1)]              0         []                            
                                                                                                  
 flow (Functional)           (None, 3000, 256)            36096     ['input_2[0][0]']             
                                                                                                  
 spo2 (Functional)           (None, 30, 256)              36096     ['input_4[0][0]']             
                                                                                              

In [11]:
annotations =et.parse("00000999-100507.rml").getroot()
events = []
samp_rate=100
for event in annotations.iter('{http://www.respironics.com/PatientStudy.xsd}Event'):
    events.append(event.attrib)
events_df= pd.DataFrame(events)
events_df.drop(['Machine', 'OriginatedOnDevice'], axis='columns', inplace=True)
events_df=events_df[events_df.Family=="Respiratory"].reset_index(drop=True)
events_df.Start=events_df.Start.astype('float64')
events_df.Duration=events_df.Duration.astype('float64')
events_df['end']=(events_df.Start+events_df.Duration)
events_df['window_start']=events_df.Start//30
events_df['window_end']=events_df.end//30
events_df

,Family,Type,Start,Duration,end,window_start,window_end
0,Respiratory,ObstructiveApnea,155.0,17.0,172.0,5.0,5.0
1,Respiratory,ObstructiveApnea,209.0,19.0,228.0,6.0,7.0
2,Respiratory,ObstructiveApnea,264.5,14.0,278.5,8.0,9.0
3,Respiratory,ObstructiveApnea,295.5,12.5,308.0,9.0,10.0
4,Respiratory,ObstructiveApnea,347.0,11.5,358.5,11.0,11.0
...,...,...,...,...,...,...,...
198,Respiratory,Hypopnea,14533.0,12.5,14545.5,484.0,484.0
199,Respiratory,Hypopnea,14662.5,12.0,14674.5,488.0,489.0
200,Respiratory,ObstructiveApnea,14711.5,10.0,14721.5,490.0,490.0
201,Respiratory,Hypopnea,14758.0,12.5,14770.5,491.0,492.0


In [12]:
events_set=set(events_df.window_start)|set(events_df.window_end)
lbls=[]
for i in range(len(ch1)):
    if i in events_set:
        lbls.append(1)
    else:
        lbls.append(0)

In [13]:
lbls=np.array(lbls)

In [12]:
X_train,X_test,y_train,y_test = [ch1[0:400,:,:],ch2[0:400,:,:]],[ch1[400:,:,:],ch2[400:,:,:]],lbls[0:400],lbls[400:]
# Step 3: Train Your Model
classifier_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classifier_model.fit(X_train, y_train, epochs=10, batch_size=8)

# Evaluate the model on the test set
test_loss, test_accuracy = classifier_model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_accuracy}')

Epoch 1/10
50/50 [==============================] - 12s 170ms/step - loss: 10.3376 - accuracy: 0.5200
Epoch 2/10
50/50 [==============================] - 10s 201ms/step - loss: 8.1112 - accuracy: 0.5625
Epoch 3/10
50/50 [==============================] - 10s 209ms/step - loss: 8.2147 - accuracy: 0.5650
Epoch 4/10
50/50 [==============================] - 11s 222ms/step - loss: 7.2252 - accuracy: 0.5725
Epoch 5/10
50/50 [==============================] - 13s 256ms/step - loss: 7.0691 - accuracy: 0.5750
Epoch 6/10
50/50 [==============================] - 10s 207ms/step - loss: 6.7369 - accuracy: 0.6400
Epoch 7/10
50/50 [==============================] - 10s 203ms/step - loss: 6.7027 - accuracy: 0.6300
Epoch 8/10
50/50 [==============================] - 10s 203ms/step - loss: 6.8112 - accuracy: 0.6225
Epoch 9/10
50/50 [==============================] - 10s 200ms/step - loss: 6.4559 - accuracy: 0.6550
Epoch 10/10
3/3 [==============================] - 1s 147ms/step - loss: 6.4168 - accuracy

## Contrastive loss

In [14]:
def cocoa_loss(ytrue, ypred):
    temperature = 0.5
    lambd=3.9e-3
    scale_loss=1/32
    batch_size, dim_size = ypred.shape[1], ypred.shape[0]
    print(ypred.shape)
    # Positive Pairs
    pos_error = []
    for i in range(batch_size):
        sim = tf.linalg.matmul(ypred[:, i, :], ypred[:, i, :], transpose_b=True)
        sim = tf.subtract(tf.ones([dim_size, dim_size], dtype=tf.dtypes.float32), sim)
        sim = tf.exp(sim/temperature)
        pos_error.append(tf.reduce_mean(sim))
    # Negative pairs
    neg_error = 0
    for i in range(dim_size):
        sim = tf.cast(tf.linalg.matmul(ypred[i], ypred[i], transpose_b=True), dtype=tf.dtypes.float32)
        sim = tf.exp(sim /temperature)
        # sim = tf.add(sim, tf.ones([batch_size, batch_size]))
        tri_mask = np.ones(batch_size ** 2, dtype=bool).reshape(batch_size, batch_size)
        tri_mask[np.diag_indices(batch_size)] = False
        off_diag_sim = tf.reshape(tf.boolean_mask(sim, tri_mask), [batch_size, batch_size - 1])
        neg_error += (tf.reduce_mean(off_diag_sim, axis=-1))

    error = tf.multiply(tf.reduce_sum(pos_error),scale_loss) + lambd * tf.reduce_sum(neg_error)

    return error

# ------------------------------------------------------------------------- #
class DotProduct(tf.keras.layers.Layer):
    def call(self, x, y):
        x = tf.nn.l2_normalize(x, axis=-1)
        y = tf.nn.l2_normalize(y, axis=-1)
        return tf.linalg.matmul(x, y, transpose_b=True)


# ------------------------------------------------------------------------- #
class ContrastiveModel(tf.keras.Model):
    def __init__(self, embedding_model, loss_fn, temperature=1.0, **kwargs):
        super().__init__()
        self.embedding_model = embedding_model
        self._temperature = temperature
        self._similarity_layer = DotProduct()
        self._lossfn = loss_fn

    def train_step(self, data):
        with tf.GradientTape() as tape:
            modality_embeddings = self.embedding_model(data, training=True)

            sparse_labels = tf.range(tf.shape(modality_embeddings[0])[0])

            pred = modality_embeddings
            pred = tf.nn.l2_normalize(tf.stack(pred), axis=-1)

            loss = self.compiled_loss(sparse_labels, pred)
            loss += sum(self.losses)

        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        return {m.name: m.result() for m in self.metrics}

    def call(self, input):
        return self.embedding_model(input)


In [15]:
ssl = ContrastiveModel(embedding_model, cocoa_loss, 1/32)

In [24]:
input_shape

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 3000, 1)]            0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 30, 1)]              0         []                            
                                                                                                  
 flow (Functional)           (None, 3000, 256)            36096     ['input_2[0][0]']             
                                                                                                  
 spo2 (Functional)           (None, 30, 256)              36096     ['input_4[0][0]']             
                                                                                              

In [17]:
X_train,X_test,y_train,y_test = [ch1[0:400,:,:],ch2[0:400,:,:]],[ch1[400:,:,:],ch2[400:,:,:]],[lbls[0:400],lbls[0:400]],[lbls[400:],lbls[400:]]
# Step 3: Train Your Model

ssl.compile(optimizer='adam', loss=cocoa_loss, metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
result = ssl.fit(X_train, epochs=10, batch_size=8)


Epoch 1/10
(2, 8, 256)
(2, 8, 256)
50/50 [==============================] - 32s 515ms/step - loss: 0.6943
Epoch 2/10
50/50 [==============================] - 26s 528ms/step - loss: 0.6530
Epoch 3/10
50/50 [==============================] - 31s 628ms/step - loss: 0.6522
Epoch 4/10
50/50 [==============================] - 25s 503ms/step - loss: 0.6524
Epoch 5/10
50/50 [==============================] - 28s 552ms/step - loss: 0.6518
Epoch 6/10
50/50 [==============================] - 24s 471ms/step - loss: 0.6508
Epoch 7/10
50/50 [==============================] - 29s 591ms/step - loss: 0.6517
Epoch 8/10
50/50 [==============================] - 29s 574ms/step - loss: 0.6505
Epoch 9/10
50/50 [==============================] - 28s 562ms/step - loss: 0.6512
Epoch 10/10
50/50 [==============================] - 28s 569ms/step - loss: 0.6516


In [18]:
for layer in ssl.layers:
    print(layer.name)

model
dot_product


In [22]:
base_model=ssl
base_model=tf.keras.Model(ssl.input,ssl.output)

AttributeError: Layer contrastive_model is not connected, no input to return.

In [20]:
base_model.input

AttributeError: Layer contrastive_model is not connected, no input to return.

## add the base model 

In [45]:
base_model = ssl
for layer in base_model.layers:
    layer.trainable = False
input_x = base_model.input
xi = base_model(input_x)
x = Concatenate()(xi)
x = Flatten()(x)

x = Dense(128, activation="relu", kernel_regularizer=l1(self.reg_dense))(x)

classifier_model = Dense(
    1,
    activation="sigmoid",
    kernel_regularizer=l1(0.005),
)(x)
c_model = Model(input_x, classifier_model)
c_model.compile(
    optimizer='adam',
    loss="categorical_crossentropy",
    metrics=["categorical_accuracy"],
)

AttributeError: Layer contrastive_model_2 is not connected, no input to return.

In [38]:
ssl.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [23]:
X_train[0].shape
# y_train[1].shape

(400, 3000, 1)

In [203]:
# Create TensorFlow Datasets from your data
train_dataset = tf.data.Dataset.from_tensor_slices(([X_train_modality_1, X_train_modality_2], y_train)).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices(([X_test_modality_1, X_test_modality_2], y_test)).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

# Compile the model
classifier_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
classifier_model.fit(train_dataset, epochs=10)

# Evaluate the model
test_loss, test_accuracy = classifier_model.evaluate(test_dataset)
print(f'Test accuracy: {test_accuracy}')


(96, 3000, 1)